In [1]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import TypedDict, Dict, Any


In [2]:
class GraphState(TypedDict):
    query: str
    legal: dict
    compliance: dict
    finance: dict
    operations: dict
    memory: list


In [3]:
def legal_agent(state: GraphState):
    time.sleep(1)  # simulate processing time
    state["legal"] = {"clauses": ["Termination", "Indemnification"]}
    print("Legal agent done")
    return state

def compliance_agent(state: GraphState):
    time.sleep(1.2)
    state["compliance"] = {"clauses": ["GDPR", "Audit"]}
    print("Compliance agent done")
    return state

def finance_agent(state: GraphState):
    time.sleep(0.8)
    state["finance"] = {"clauses": ["Payment", "Interest"]}
    print("Finance agent done")
    return state

def operations_agent(state: GraphState):
    time.sleep(0.5)
    state["operations"] = {"clauses": ["Delivery", "Uptime"]}
    print("Operations agent done")
    return state


In [4]:
state_seq: GraphState = {
    "query": "Analyze contract",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {},
    "memory": []
}

# Sequential execution
start_seq = time.time()
state_seq = legal_agent(state_seq)
state_seq = compliance_agent(state_seq)
state_seq = finance_agent(state_seq)
state_seq = operations_agent(state_seq)
end_seq = time.time()

print("\nSequential Runtime:", round(end_seq - start_seq, 2), "seconds")


Legal agent done
Compliance agent done
Finance agent done
Operations agent done

Sequential Runtime: 3.5 seconds


In [5]:
state_parallel: GraphState = {
    "query": "Analyze contract",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {},
    "memory": []
}

agents = {
    "legal": legal_agent,
    "compliance": compliance_agent,
    "finance": finance_agent,
    "operations": operations_agent
}

start_par = time.time()
with ThreadPoolExecutor(max_workers=4) as executor:
    futures = {executor.submit(func, state_parallel.copy()): name for name, func in agents.items()}
    for future in as_completed(futures):
        name = futures[future]
        result = future.result()
        state_parallel[name] = result[name]

end_par = time.time()

print("\nParallel Runtime:", round(end_par - start_par, 2), "seconds")



Operations agent done
Finance agent done
Legal agent done
Compliance agent done

Parallel Runtime: 1.22 seconds


In [6]:
print("\nSequential State Output:")
print(state_seq)

print("\nParallel State Output:")
print(state_parallel)



Sequential State Output:
{'query': 'Analyze contract', 'legal': {'clauses': ['Termination', 'Indemnification']}, 'compliance': {'clauses': ['GDPR', 'Audit']}, 'finance': {'clauses': ['Payment', 'Interest']}, 'operations': {'clauses': ['Delivery', 'Uptime']}, 'memory': []}

Parallel State Output:
{'query': 'Analyze contract', 'legal': {'clauses': ['Termination', 'Indemnification']}, 'compliance': {'clauses': ['GDPR', 'Audit']}, 'finance': {'clauses': ['Payment', 'Interest']}, 'operations': {'clauses': ['Delivery', 'Uptime']}, 'memory': []}


In [ ]:
#Persisting Agent Outputs into Memory

In [7]:
import time
from datetime import datetime
from typing import List, Dict, Any
# Simulate vector embedding function
def embed_text(text: str) -> List[float]:
    """Dummy embedding function."""
    return [float(ord(c)) for c in text[:10]]  # simple numeric vector for illustration


In [8]:
# Example outputs from multiple agents
agent_outputs = {
    "legal": {"clauses": ["Termination clause", "Indemnification clause"]},
    "compliance": {"clauses": ["GDPR compliance", "Audit reporting"]},
    "finance": {"clauses": ["Payment terms", "Interest penalties"]},
    "operations": {"clauses": ["Delivery timeline", "Uptime guarantee"]}
}

contract_id = "CONTRACT_1234"


In [9]:
vector_records = []

for agent_name, output in agent_outputs.items():
    for clause in output["clauses"]:
        record = {
            "text": clause,
            "metadata": {
                "agent": agent_name,
                "contract_id": contract_id,
                "timestamp": datetime.now().isoformat()
            },
            "embedding": embed_text(clause)  # vector embedding
        }
        vector_records.append(record)


In [10]:
# Simulate a vector DB as a list
vector_db: List[Dict[str, Any]] = []

# Store all records
vector_db.extend(vector_records)

print(f"Stored {len(vector_db)} records in vector DB")


Stored 8 records in vector DB


In [11]:
print("\nSample Stored Record:")
print(vector_db[0])  # show first record



Sample Stored Record:
{'text': 'Termination clause', 'metadata': {'agent': 'legal', 'contract_id': 'CONTRACT_1234', 'timestamp': '2026-01-14T00:43:29.423152'}, 'embedding': [84.0, 101.0, 114.0, 109.0, 105.0, 110.0, 97.0, 116.0, 105.0, 111.0]}


In [ ]:
#Querying Stored Agent Memory (Recall & Reuse)

In [12]:
def query_memory(vector_db, agent=None):
    """
    Retrieve stored agent outputs.
    If agent is None → return all records.
    """
    if agent:
        return [r for r in vector_db if r["metadata"]["agent"] == agent]
    return vector_db


In [13]:
legal_memory = query_memory(vector_db, agent="legal")

print("Legal Memory:")
for record in legal_memory:
    print(record)


Legal Memory:
{'text': 'Termination clause', 'metadata': {'agent': 'legal', 'contract_id': 'CONTRACT_1234', 'timestamp': '2026-01-14T00:43:29.423152'}, 'embedding': [84.0, 101.0, 114.0, 109.0, 105.0, 110.0, 97.0, 116.0, 105.0, 111.0]}
{'text': 'Indemnification clause', 'metadata': {'agent': 'legal', 'contract_id': 'CONTRACT_1234', 'timestamp': '2026-01-14T00:43:29.423177'}, 'embedding': [73.0, 110.0, 100.0, 101.0, 109.0, 110.0, 105.0, 102.0, 105.0, 99.0]}


In [14]:
print("\nLegal Clauses Retrieved:", len(legal_memory))



Legal Clauses Retrieved: 2


In [15]:
finance_memory = query_memory(vector_db, agent="finance")

print("\nFinance Memory:")
for record in finance_memory:
    print(record)



Finance Memory:
{'text': 'Payment terms', 'metadata': {'agent': 'finance', 'contract_id': 'CONTRACT_1234', 'timestamp': '2026-01-14T00:43:29.423202'}, 'embedding': [80.0, 97.0, 121.0, 109.0, 101.0, 110.0, 116.0, 32.0, 116.0, 101.0]}
{'text': 'Interest penalties', 'metadata': {'agent': 'finance', 'contract_id': 'CONTRACT_1234', 'timestamp': '2026-01-14T00:43:29.423208'}, 'embedding': [73.0, 110.0, 116.0, 101.0, 114.0, 101.0, 115.0, 116.0, 32.0, 112.0]}


In [16]:
all_memory = query_memory(vector_db)

print("\nAll Retrieved Memory:")
for record in all_memory:
    print(record)



All Retrieved Memory:
{'text': 'Termination clause', 'metadata': {'agent': 'legal', 'contract_id': 'CONTRACT_1234', 'timestamp': '2026-01-14T00:43:29.423152'}, 'embedding': [84.0, 101.0, 114.0, 109.0, 105.0, 110.0, 97.0, 116.0, 105.0, 111.0]}
{'text': 'Indemnification clause', 'metadata': {'agent': 'legal', 'contract_id': 'CONTRACT_1234', 'timestamp': '2026-01-14T00:43:29.423177'}, 'embedding': [73.0, 110.0, 100.0, 101.0, 109.0, 110.0, 105.0, 102.0, 105.0, 99.0]}
{'text': 'GDPR compliance', 'metadata': {'agent': 'compliance', 'contract_id': 'CONTRACT_1234', 'timestamp': '2026-01-14T00:43:29.423187'}, 'embedding': [71.0, 68.0, 80.0, 82.0, 32.0, 99.0, 111.0, 109.0, 112.0, 108.0]}
{'text': 'Audit reporting', 'metadata': {'agent': 'compliance', 'contract_id': 'CONTRACT_1234', 'timestamp': '2026-01-14T00:43:29.423194'}, 'embedding': [65.0, 117.0, 100.0, 105.0, 116.0, 32.0, 114.0, 101.0, 112.0, 111.0]}
{'text': 'Payment terms', 'metadata': {'agent': 'finance', 'contract_id': 'CONTRACT_1234'

In [17]:
def compare_risks(memory_records):
    """
    Compare risk by counting clauses per agent.
    More clauses = higher potential risk.
    """
    risk_counter = {}

    for record in memory_records:
        agent = record["metadata"]["agent"]
        risk_counter[agent] = risk_counter.get(agent, 0) + 1

    return risk_counter

risk_comparison = compare_risks(all_memory)

print("\nRisk Comparison Across Agents:")
for agent, count in risk_comparison.items():
    print(f"{agent}: {count} clauses")



Risk Comparison Across Agents:
legal: 2 clauses
compliance: 2 clauses
finance: 2 clauses
operations: 2 clauses


In [18]:
print("\nUsing stored memory instead of re-running agents...")
for record in all_memory:
    print(f"[{record['metadata']['agent'].upper()}] {record['text']}")



Using stored memory instead of re-running agents...
[LEGAL] Termination clause
[LEGAL] Indemnification clause
[COMPLIANCE] GDPR compliance
[COMPLIANCE] Audit reporting
[FINANCE] Payment terms
[FINANCE] Interest penalties
[OPERATIONS] Delivery timeline
[OPERATIONS] Uptime guarantee


In [19]:
#Cross-Agent Refinement

In [21]:
all_agent_memory = query_memory(vector_db)


In [24]:
for m in all_agent_memory:
    meta = m["metadata"]
    if "risk_level" not in meta:
        meta["risk_level"] = "medium"   # default baseline risk


In [25]:
latest_by_agent = {}

for m in all_agent_memory:
    agent = m["metadata"].get("agent")
    if agent:
        latest_by_agent[agent] = m["metadata"]


In [26]:
shared_context = "\n".join([
    f"{agent} risk: {meta['risk_level']}"
    for agent, meta in latest_by_agent.items()
])

print(shared_context)


legal risk: medium
compliance risk: medium
finance risk: medium
operations risk: medium


In [27]:
finance_memory = [
    m for m in vector_db
    if m["metadata"]["agent"] == "finance"
]

latest_finance = finance_memory[-1]["metadata"]
print("Finance Output Seen by Compliance:", latest_finance)


Finance Output Seen by Compliance: {'agent': 'finance', 'contract_id': 'CONTRACT_1234', 'timestamp': '2026-01-14T00:43:29.423208', 'risk_level': 'medium'}


In [28]:
def compliance_refinement_agent(shared_context, finance_output):
    if finance_output["risk_level"] in ["medium", "high"]:
        return {
            "risk_level": "high",
            "reason": "Financial penalties increase GDPR and audit exposure",
            "confidence": 0.88
        }
    return {
        "risk_level": "medium",
        "reason": "No financial impact on compliance",
        "confidence": 0.70
    }

refined_compliance = compliance_refinement_agent(
    shared_context,
    latest_finance
)

print(refined_compliance)


{'risk_level': 'high', 'reason': 'Financial penalties increase GDPR and audit exposure', 'confidence': 0.88}


In [29]:
vector_db.append({
    "text": refined_compliance["reason"],
    "metadata": {
        "agent": "compliance",
        "risk_level": refined_compliance["risk_level"],
        "confidence": refined_compliance["confidence"],
        "timestamp": datetime.now().isoformat()
    }
})


In [30]:
latest_compliance = [
    m for m in vector_db if m["metadata"]["agent"] == "compliance"
][-1]["metadata"]

print("Updated Compliance Risk:", latest_compliance)


Updated Compliance Risk: {'agent': 'compliance', 'risk_level': 'high', 'confidence': 0.88, 'timestamp': '2026-01-14T00:51:05.220307'}


In [31]:
#Final Contract-Level JSON Output

In [32]:
FINAL_CONTRACT_SCHEMA = {
    "contract_id": "",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {},
    "overall_risk": "",
    "average_confidence": 0.0,
    "high_risk_clauses": [],
    "generated_at": ""
}


In [33]:
def get_latest_agent_output(agent):
    res = query_agent_memory(
        query="risk",
        agent=agent,
        top_k=1
    )
    return res.matches[0].metadata if res.matches else {}


In [39]:
def query_agent_memory(query=None, agent=None, top_k=5):
    """
    Query in-memory agent outputs stored as a list
    """
    matches = []

    for record in vector_db:
        metadata = record.get("metadata", {})

        if agent and metadata.get("agent") != agent:
            continue

        if query and query.lower() not in record.get("text", "").lower():
            continue

        matches.append(record)

    # return most recent entries
    matches = sorted(
        matches,
        key=lambda x: x.get("metadata", {}).get("timestamp", ""),
        reverse=True
    )[:top_k]

    return matches


In [40]:
def get_latest_agent_output(agent):
    res = query_agent_memory(
        query="risk",
        agent=agent,
        top_k=1
    )
    return res[0]["metadata"] if res else {}


In [41]:
legal_out = get_latest_agent_output("legal")
compliance_out = get_latest_agent_output("compliance")
finance_out = get_latest_agent_output("finance")
operations_out = get_latest_agent_output("operations")



In [42]:
vector_db.append({
    "text": "Legal agent risk analysis",
    "metadata": {
        "agent": "legal",
        "risk_level": "medium",
        "confidence": 0.72,
        "timestamp": "2026-01-14T23:10:00"
    }
})


In [ ]:
#Report Template Design (Human-Readable Output)

In [43]:
REPORT_STRUCTURE = [
    "Executive Summary",
    "Overall Risk Assessment",
    "Legal Analysis",
    "Compliance Analysis",
    "Financial Analysis",
    "Operational Analysis",
    "Conclusion & Recommendations"
]


In [44]:
def build_executive_summary(final_json):
    return (
        f"This contract shows an overall risk level of "
        f"{final_json.get('overall_risk', 'unknown')}. "
        "Key risks come from legal, financial, and operational clauses. "
        "Careful review is recommended before approval."
    )


In [45]:
def build_section(title, content):
    bullets = []

    if not content:
        bullets.append("- No significant risks identified.")
    else:
        for k, v in content.items():
            bullets.append(f"- {k.replace('_', ' ').title()}: {v}")

    return {
        "title": title,
        "bullets": bullets
    }


In [46]:
def generate_report(final_json):
    report = {}

    report["Executive Summary"] = build_executive_summary(final_json)

    report["Overall Risk Assessment"] = [
        f"- Overall risk level: {final_json.get('overall_risk', 'unknown')}"
    ]

    report["Legal Analysis"] = build_section(
        "Legal Analysis",
        final_json.get("legal", {})
    )["bullets"]

    report["Compliance Analysis"] = build_section(
        "Compliance Analysis",
        final_json.get("compliance", {})
    )["bullets"]

    report["Financial Analysis"] = build_section(
        "Financial Analysis",
        final_json.get("finance", {})
    )["bullets"]

    report["Operational Analysis"] = build_section(
        "Operational Analysis",
        final_json.get("operations", {})
    )["bullets"]

    report["Conclusion & Recommendations"] = [
        "- Review high-risk clauses carefully",
        "- Negotiate unclear obligations",
        "- Proceed only after legal approval"
    ]

    return report


In [48]:
final_contract_json = {
    "contract_id": "C-001",
    "legal": {"risk_level": "medium"},
    "compliance": {"risk_level": "medium"},
    "finance": {"risk_level": "medium"},
    "operations": {"risk_level": "medium"},
    "overall_risk": "medium",
    "generated_at": "2026-01-14"
}

human_report = generate_report(final_contract_json)

print("Executive Summary:")
print(human_report["Executive Summary"])


Executive Summary:
This contract shows an overall risk level of medium. Key risks come from legal, financial, and operational clauses. Careful review is recommended before approval.


In [ ]:
#Report Formatting & Tone Customization

In [50]:
TONE_TEMPLATES = {
    "executive": {
        "title_prefix": "🔴 HIGH RISK:" ,
        "bullet": "•",
    },
    "neutral": {
        "title_prefix": "⚠",
        "bullet": "-",
    }
}

	

In [51]:
def is_high_risk(section_data):
    return section_data.get("risk_level") == "high"


In [52]:
def format_section(title, section_data, tone="executive"):
    template = TONE_TEMPLATES[tone]
    bullet = template["bullet"]

    header = title
    if is_high_risk(section_data):
        header = f"{template['title_prefix']} {title}"

    lines = [header]

    for key, value in section_data.items():
        lines.append(f"{bullet} {key.replace('_',' ').title()}: {value}")

    return "\n".join(lines)


In [53]:
def format_report(final_contract_json, tone="executive"):
    report = {}

    for section in ["legal", "compliance", "finance", "operations"]:
        report[section.title()] = format_section(
            section.title(),
            final_contract_json[section],
            tone
        )

    return report


In [54]:
executive_report = format_report(final_contract_json, tone="executive")

for section, content in executive_report.items():
    print("\n" + content)



Legal
• Risk Level: medium

Compliance
• Risk Level: medium

Finance
• Risk Level: medium

Operations
• Risk Level: medium


In [ ]:
#FastAPI Backend for Contract Analysis

In [58]:
from fastapi import FastAPI, UploadFile, File, HTTPException, Query
from datetime import datetime

app = FastAPI(title="Contract Analysis API")



In [59]:
def run_full_pipeline(contract_text: str, tone: str = "executive"):
    """
    1. RAG retrieval
    2. Parallel agents
    3. Memory lookup & refinement
    4. Final JSON aggregation
    5. Report formatting
    """

    if not contract_text.strip():
        raise ValueError("Contract text is empty")

    # Replace these with your actual pipeline functions
    final_json = run_contract_pipeline(contract_text)
    formatted_report = format_report(final_json, tone)

    return final_json, formatted_report


In [60]:
def format_report(final_json, tone="executive"):
    if tone == "simple":
        return generate_simple_report(final_json)
    return generate_executive_report(final_json)
